In [1]:
import requests #type:ignore NOQA
import gspread #type:ignore NOQA
import random
from datetime import date, timedelta, datetime
from time import sleep
#import json
sa = gspread.service_account(filename="sa_creds.json")


In [3]:
#Ryanair queries:
def getDestinations(airportFrom: str):
    base = 'https://www.ryanair.com/api/locate/v5/routes'
    departureAirportCode = f'departureAirportCode={airportFrom.upper()}'
    fields = 'fields=arrivalAirport.code&fields=arrivalAirport.name&fields=arrivalAirport.seoName&fields=arrivalAirport.timeZone&fields=arrivalAirport.city.code&fields=arrivalAirport.city.name&fields=arrivalAirport.coordinates.latitude&fields=arrivalAirport.coordinates.longitude&fields=arrivalAirport.country.code&fields=arrivalAirport.country.name&fields=arrivalAirport.country.currency&fields=operator'
    return f'{base}?{departureAirportCode}&{fields}'

def getAvailabilities(airportFrom: str, airportTo: str):
    base = 'https://www.ryanair.com/api/farfnd/3/oneWayFares'
    return f'{base}/{airportFrom.upper()}/{airportTo.upper()}/availabilities'

def getFares(airportFrom: str, airportTo: str, dateOutIn: str, adult: int=1, teen: int=0, children: int=0, infant: int=0, flexDays: int=6):
    base = 'https://www.ryanair.com/api/booking/v4/en-en/availability'
    
    return f'{base}?ADT={adult}&CHD={children}&Destination={airportTo.upper()}&INF={infant}&Origin={airportFrom.upper()}&TEEN={teen}&IncludeConnectingFlights=false&DateOut={dateOutIn}&FlexDaysOut={flexDays}&DateIn={dateOutIn}&FlexDaysIn={flexDays}&RoundTrip=true&ToUs=AGREED'

def readDestinations(trip, mappingFor, destinationsJSON, destinationsList):
#read destinations from JSON into list
    for d in destinationsJSON:
        dArrivalAirportCode = d['arrivalAirport']['code']
        dAirportName = d['arrivalAirport']['name']
        dAirportSeoName = d['arrivalAirport']['seoName']
        dAirportCountryCode = d['arrivalAirport']['country']['code']
        dAirportCountryName = d['arrivalAirport']['country']['name']
        dAirportCityName = d['arrivalAirport']['city']['name']
        dAirportTimeZone = d['arrivalAirport']['timeZone']
        dCurrency = d['arrivalAirport']['country']['currency']
        dlatitude = str(d['arrivalAirport']['coordinates']['latitude'])
        dlongitude = str(d['arrivalAirport']['coordinates']['longitude'])
        dDateChecked = str(date.today())
        destinationsList.append([trip, mappingFor, dArrivalAirportCode, dAirportName, dAirportSeoName, dAirportCountryCode, dAirportCountryName, dAirportCityName, dAirportTimeZone, dCurrency, dlatitude, dlongitude, dDateChecked])

def buildFareSearch(activeTrips, ryanDestinations):
#gather destinations and dates for active trips
    for f in activeTrips:
        for i in ryanDestinations:
            if f[0] in i:
                fareDestinations.append([f[0], f[1], f[2], i[1], i[2], f[3]])

def buildFlightDate(flightDate: str, flightFrom: str, flightTo: str):
#make date range and select closest date from available dates
    fdate = datetime.strptime(flightDate,"%Y-%m-%d").date()
    if fdate > date.today() + timedelta(days=4):
        fdate = fdate - timedelta(days=3)
    availables = requests.get(getAvailabilities(flightFrom,flightTo)).json()
    if availables == []:
        fdate = 'NoFlights!'
    else:        
        for e, a in enumerate(availables):
            if datetime.strptime(a,"%Y-%m-%d").date() < fdate:
                availables[e] = 999
            else:
                availables[e] = abs((datetime.strptime(a,"%Y-%m-%d").date()-fdate).days)
        fdate = (fdate + timedelta(days=min(availables))).strftime("%Y-%m-%d")
    return fdate

rDestinations = []
fareDestinations = []

In [4]:
#getting all worksheets on sheet
wksheets = []
sh = sa.open("Flights")

sh_md = sh.fetch_sheet_metadata()
for s in sh_md['sheets']:
    wksheets.append(s['properties']['title'])
# wksheets

In [5]:
#getting travel route and dates
wks = sh.worksheet('Flights')
flightSearch_all = wks.get_all_values()[1:]

#adding trip to front
for f in flightSearch_all:
    f.insert(0, f'{f[0]}-{f[1]}_{f[2]}')

#getting active trips only
flightSearch_active = [f for f in flightSearch_all if 'TRUE' in f]

#identifying trips that are already in RyanDestinations worksheet
tripsAlreadyIn = []
wks_dest = sh.worksheet('RyanDestinations')
tripsIn = wks_dest.get()
tripsIn = tripsIn[1:]
for f in tripsIn:
    if f[0] not in tripsAlreadyIn:
        tripsAlreadyIn.append(f[0])

#getting list of only new trips
flightSearch_activeNew = [f for f in flightSearch_active if f[0] not in tripsAlreadyIn]

In [7]:
tripsAlreadyIn

['VNO-PSR_2022-12-20']

In [6]:
#adding new trips to RyanDestinations worksheet
if len(flightSearch_activeNew) > 0:
    for f in flightSearch_activeNew:
        fromMappingFor = f[1]
        toMappingFor = f[2]
        theTrip = f[0]
#Retrieving destinations for Origin airport and apprnding to rDestinations[]
        destinations = requests.get(getDestinations(fromMappingFor)).json()
        readDestinations(theTrip, fromMappingFor, destinations, rDestinations)
        sleep(random.randint(1, 5))
#Retrieving destinations for final Destination airport and apprnding to rDestinations[]
        destinations = requests.get(getDestinations(toMappingFor)).json()
        readDestinations(theTrip, toMappingFor, destinations, rDestinations)
        sleep(random.randint(1, 5))
#Sendin all destinations to GSheets 'RyanDestinations' sheet
    if len(rDestinations) > 0:
        wks_dest.append_rows(rDestinations)

In [50]:
dest = 'https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=BGY&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2022-12-18&FlexDaysOut=6&DateIn=2022-12-18&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED'

fares = requests.get(dest).json()

In [73]:
# fares['currency']
# for i in fares['trips']:
#     print(i['origin'],i['destination'])

for i in fares['trips']:
    for f in i['dates']:
        # print(f['dateOut'])
        for d in f['flights']:
            print(d['flightKey'])
            # print(d['regularFare'])
# for f in fares['trips'][0]:
#     print(f)

FR~2871~ ~~VNO~12/18/2022 13:25~BGY~12/18/2022 15:00~~
FR~2871~ ~~VNO~12/19/2022 14:10~BGY~12/19/2022 15:45~~
FR~2871~ ~~VNO~12/20/2022 14:20~BGY~12/20/2022 15:55~~
FR~2871~ ~~VNO~12/22/2022 15:05~BGY~12/22/2022 16:40~~
FR~2871~ ~~VNO~12/23/2022 21:40~BGY~12/23/2022 23:15~~
FR~2871~ ~~VNO~12/24/2022 11:05~BGY~12/24/2022 12:40~~
FR~2872~ ~~BGY~12/18/2022 15:25~VNO~12/18/2022 18:50~~
FR~2872~ ~~BGY~12/19/2022 16:10~VNO~12/19/2022 19:35~~
FR~2872~ ~~BGY~12/20/2022 10:30~VNO~12/20/2022 13:55~~
FR~2872~ ~~BGY~12/22/2022 11:15~VNO~12/22/2022 14:40~~
FR~2872~ ~~BGY~12/23/2022 17:50~VNO~12/23/2022 21:15~~
FR~2872~ ~~BGY~12/24/2022 07:15~VNO~12/24/2022 10:40~~


In [8]:
#flight fare search dates
for f in flightSearch_active:
    for t in tripsIn:
        if f[0] in t:
            fDate = buildFlightDate(f[3], t[1], t[2])
            if fDate != 'NoFlights!':
                print(getFares(t[1], t[2],fDate))
            sleep(random.randint(1, 5))

https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=ATH&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2023-03-26&FlexDaysOut=6&DateIn=2023-03-26&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=BCN&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2022-12-18&FlexDaysOut=6&DateIn=2022-12-18&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=BER&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2022-12-17&FlexDaysOut=6&DateIn=2022-12-17&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=BGY&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2022-12-18&FlexDaysOut=6&DateIn=2022-12-18&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&De

KeyboardInterrupt: 